In [63]:
import pandas as pd
import json,natsort
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [64]:
hw_data_file = './combined_uarch.csv'
high_level_data_file = './all_config_stacked.csv'
mapping_file = './mapping_funcs.json'

In [65]:
# read data as csv
hw_df = pd.read_csv(hw_data_file, index_col=0)
high_level_df = pd.read_csv(high_level_data_file)

In [66]:
hw_df.head()

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local Memory,Remote Memory,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,ImagingResampleVertical_8bpc,614.306,1.780401e+14,4.337152e+11,0.000000e+00,1.951718e+12,0.000000e+00,4.337152e+11,1.951718e+12,0.000000e+00,0.000000e+00,0.000000e+00,2.168576e+11,3.773322e+13,3.390447,4
1,ImagingResampleHorizontal_8bpc,385.980,1.040783e+14,4.920960e+11,0.000000e+00,8.611680e+11,0.000000e+00,2.460480e+11,1.107216e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.759243e+13,2.130282,4
2,at::native::copy_kernel(float),358.990,6.974640e+12,1.679080e+12,6.458000e+11,1.097860e+13,1.291600e+13,1.265768e+13,1.021656e+14,1.291600e+14,1.679080e+12,5.166400e+11,1.154690e+14,7.103800e+12,1.981320,4
3,decode_mcu,357.255,3.390039e+13,6.625203e+12,3.510497e+13,1.471311e+13,1.720832e+11,8.604160e+10,6.797286e+12,0.000000e+00,0.000000e+00,0.000000e+00,8.604160e+10,6.194995e+12,1.971744,4
4,THFloatStorage_copy,319.535,3.630283e+13,4.116816e+12,3.742560e+12,0.000000e+00,4.491072e+12,1.871280e+12,4.580893e+13,6.444688e+13,7.485120e+13,4.326399e+13,2.245536e+12,1.669182e+13,1.763562,4


In [67]:
high_level_df.head()

,config,Collation,Load,Normalize,RandomHorizontalFlip,Resize,ToTensor
0,4,433.549978,1125.375266,915.226421,61.015711,1106.893002,791.916118


In [68]:
# make config column the index
high_level_df = high_level_df.set_index('config')

# map config column with format 'b1024_gpu4_dataloader8' to '8' casted to int
# hw_df['config'] = hw_df['config'].map(lambda x: int(x.split('_')[-1].replace('dataloader', '')))

In [69]:
print('Numbers are %s (not multiplied by 100)')
high_level_df.head()

Numbers are %s (not multiplied by 100)


,Collation,Load,Normalize,RandomHorizontalFlip,Resize,ToTensor
config,,,,,,
4,433.549978,1125.375266,915.226421,61.015711,1106.893002,791.916118


In [70]:
hw_df.head()

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local Memory,Remote Memory,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,ImagingResampleVertical_8bpc,614.306,1.780401e+14,4.337152e+11,0.000000e+00,1.951718e+12,0.000000e+00,4.337152e+11,1.951718e+12,0.000000e+00,0.000000e+00,0.000000e+00,2.168576e+11,3.773322e+13,3.390447,4
1,ImagingResampleHorizontal_8bpc,385.980,1.040783e+14,4.920960e+11,0.000000e+00,8.611680e+11,0.000000e+00,2.460480e+11,1.107216e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.759243e+13,2.130282,4
2,at::native::copy_kernel(float),358.990,6.974640e+12,1.679080e+12,6.458000e+11,1.097860e+13,1.291600e+13,1.265768e+13,1.021656e+14,1.291600e+14,1.679080e+12,5.166400e+11,1.154690e+14,7.103800e+12,1.981320,4
3,decode_mcu,357.255,3.390039e+13,6.625203e+12,3.510497e+13,1.471311e+13,1.720832e+11,8.604160e+10,6.797286e+12,0.000000e+00,0.000000e+00,0.000000e+00,8.604160e+10,6.194995e+12,1.971744,4
4,THFloatStorage_copy,319.535,3.630283e+13,4.116816e+12,3.742560e+12,0.000000e+00,4.491072e+12,1.871280e+12,4.580893e+13,6.444688e+13,7.485120e+13,4.326399e+13,2.245536e+12,1.669182e+13,1.763562,4


In [71]:
# load the mapping functions
mapping_funcs = json.load(open(mapping_file))


In [72]:
mapping_funcs

{'op_to_func': {'BatchCollator': ['munmap|libc.so.6',
   'gomp_team_end|libgomp.so.1',
   'c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::TensorIteratorBase::loop_2d_from_1d<at::native::(anonymous namespace)::cpu_kernel_vec<(bool)1, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(float)#1}, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(at::vec::(anonymous namespace)::Vectorized<float>)#2}>(void, at::TensorIteratorBase&, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(float)#1}&&, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{

In [73]:
# print columns of hw_df
hw_df.columns

Index(['Function', 'CPU Time (s)', 'Retiring', 'Front-End Bound',
       'Bad Speculation', 'L1 Bound', 'L2 Bound', 'L3 Bound',
       'Memory Bandwidth', 'Local Memory', 'Remote Memory', 'Remote Cache',
       'Store Bound', 'Core Bound', 'CPU Time %', 'config'],
      dtype='object')

In [74]:
hw_columns_to_su = ['CPU Time (s)', 'Retiring', 'Front-End Bound',
       'Bad Speculation', 'L1 Bound', 'L2 Bound', 'L3 Bound',
       'Memory Bandwidth', 'Local Memory', 'Remote Memory', 'Remote Cache',
       'Store Bound', 'Core Bound', 'CPU Time %']
combined_df = pd.DataFrame()
# loop config column in high_level_df
for config in high_level_df.index:
    print('Config: ',config)
    config_df = pd.DataFrame()
    # loop through each column in high_level_df
    for python_func in high_level_df.columns:
        print('\tPython func: ',python_func)
        # get the mapping function for the python_func
        if python_func == "Collation":
            c_cppfuncs = mapping_funcs['op_to_func']['BatchCollator']
        else:
            c_cppfuncs = mapping_funcs['op_to_func'][python_func]
        python_df = pd.DataFrame()
        # loop through each column in mapping_func
        for c in c_cppfuncs:
            # rename the c func if it exists in the func_rename dict
            func_only = c.split('|')[0]
            print('\t\tC/C++ func: ',func_only)
            # get the column in hw_df that matches the c func and config
            hw_df_col = hw_df[(hw_df['config'] == config) & (hw_df['Function'] == func_only)].copy()
            # check if empty
            if hw_df_col.empty:
                print('\t\t\tEmpty')
                continue
            # print('\t\t\tHW data: ',hw_df_col)
            # drop the config and function column
            hw_df_col = hw_df_col.drop(columns=['config', 'Function'])
            # multiply the hw_df_col by the percentage of the python_func if its a common op
            if c in mapping_funcs['common_funcs_to_op']:
                ops = mapping_funcs['common_funcs_to_op'][c]
                base_total = 0
                for op in ops:
                    base_total += high_level_df.loc[config, op]
                scale_factor = high_level_df.loc[config, python_func] / base_total
                print('\t\t\tScale factor: ',scale_factor)
                hw_df_col = hw_df_col.mul(scale_factor)
            # concat the hw_df_col to the python_df
            python_df = pd.concat([python_df, hw_df_col])
        # sum the python_df and print
        python_series = python_df.sum()
        python_df = pd.DataFrame({python_func: python_series})
        # concat the python_df to the config_df
        config_df = pd.concat([config_df, python_df],axis=1)

    # transpose the config_df
    config_df = config_df.T
    # make index a column with name 'Function'
    config_df = config_df.reset_index().rename(columns={'index': 'Function'})
    # add config column to config_df
    config_df['config'] = config
    # concat the config_df to the combined_df
    combined_df = pd.concat([combined_df, config_df])
# reset the index
combined_df = combined_df.reset_index(drop=True)

Config:  4
	Python func:  Collation
		C/C++ func:  munmap
			Empty
		C/C++ func:  gomp_team_end
			Empty
		C/C++ func:  c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::TensorIteratorBase::loop_2d_from_1d<at::native::(anonymous namespace)::cpu_kernel_vec<(bool)1, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(float)#1}, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(at::vec::(anonymous namespace)::Vectorized<float>)#2}>(void, at::TensorIteratorBase&, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(float)#1}&&, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#

In [75]:
combined_df.head()

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local Memory,Remote Memory,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,Collation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,Load,1026.769265,1.234057e+14,2.749928e+13,5.283827e+13,2.726350e+13,1.897129e+12,6.425893e+12,2.752266e+13,6.931421e+11,0.000000e+00,0.0,1.393110e+13,2.649822e+13,5.666894,4
2,Normalize,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
3,RandomHorizontalFlip,35.519053,9.908453e+12,4.390892e+11,2.418705e+10,4.760800e+11,2.341120e+10,2.557468e+11,6.269831e+12,2.233798e+12,1.934964e+10,0.0,2.933311e+11,6.584474e+11,0.196035,4
4,Resize,1037.367406,2.856161e+14,6.767853e+12,4.387800e+11,4.229519e+12,0.000000e+00,2.134005e+12,5.315517e+12,7.396577e+12,3.510240e+11,0.0,2.293942e+11,5.622829e+13,5.725387,4


In [76]:
# print columns of combined_df
combined_df.columns

Index(['Function', 'CPU Time (s)', 'Retiring', 'Front-End Bound',
       'Bad Speculation', 'L1 Bound', 'L2 Bound', 'L3 Bound',
       'Memory Bandwidth', 'Local Memory', 'Remote Memory', 'Remote Cache',
       'Store Bound', 'Core Bound', 'CPU Time %', 'config'],
      dtype='object')

In [77]:
combined_df.head(len(combined_df))

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local Memory,Remote Memory,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,Collation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,Load,1026.769265,1.234057e+14,2.749928e+13,5.283827e+13,2.726350e+13,1.897129e+12,6.425893e+12,2.752266e+13,6.931421e+11,0.000000e+00,0.0,1.393110e+13,2.649822e+13,5.666894,4
2,Normalize,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
3,RandomHorizontalFlip,35.519053,9.908453e+12,4.390892e+11,2.418705e+10,4.760800e+11,2.341120e+10,2.557468e+11,6.269831e+12,2.233798e+12,1.934964e+10,0.0,2.933311e+11,6.584474e+11,0.196035,4
4,Resize,1037.367406,2.856161e+14,6.767853e+12,4.387800e+11,4.229519e+12,0.000000e+00,2.134005e+12,5.315517e+12,7.396577e+12,3.510240e+11,0.0,2.293942e+11,5.622829e+13,5.725387,4
5,ToTensor,140.365276,2.068976e+13,8.208828e+12,2.279578e+12,5.431247e+12,1.291311e+12,4.916942e+12,1.656600e+13,6.568854e+12,2.511368e+11,0.0,8.494394e+12,1.878772e+12,0.774697,4


In [78]:
for col in combined_df.columns:
    if col == 'config' or col == 'Function':
        continue
    print(col)
    # pivot table
    df_pivot = combined_df.pivot(index='config', columns='Function', values=col)
    # sort the index using natsort
    df_pivot = df_pivot.reindex(natsort.natsorted(df_pivot.index))
    # find lowest value in entire dataframe and normalize the dataframe by dividing each value by the lowest value suhc that its not 0
    ax = df_pivot.plot(kind='bar', stacked=True, figsize=(12,6),colormap='jet')
    # for c in ax.containers:
    #     ax.bar_label(c, label_type='center')
    # make x axis label horizontal
    plt.xticks(rotation=0)
    # add x axis label
    plt.xlabel('Dataloaders')
    # add y axis label
    plt.ylabel(f"{col}")
    # add title to legend
    plt.title(f"Hardware metric for preprocessing operations")    
    # reverse legend order
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.legend(handles[::-1], labels[::-1],loc='center left', bbox_to_anchor=(1.0, 0.5))
    plt.subplots_adjust(right=0.70)
    plt.savefig(f'./mapped_funcs_metric_figs/{col}.png')
    plt.close()


CPU Time (s)


Retiring
Front-End Bound
Bad Speculation
L1 Bound
L2 Bound
L3 Bound
Memory Bandwidth
Local Memory
Remote Memory
Remote Cache
Store Bound
Core Bound
CPU Time %
